In [1]:
# lib import
import numpy as np
import pandas as pd

TRAINING = False

In [2]:
def generate_features(df):
    features = ['seconds_in_bucket', 'imbalance_buy_sell_flag',
               'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
                'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2'
               ]

    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')

    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']

    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                df[f'{a}_{b}_imb'] = df.eval(f'({a}-{b})/({a}+{b})')
                features.append(f'{a}_{b}_imb')

    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = df[[a,b,c]].max(axis=1)
                    min_ = df[[a,b,c]].min(axis=1)
                    mid_ = df[[a,b,c]].sum(axis=1)-min_-max_

                    df[f'{a}_{b}_{c}_imb2'] = (max_-mid_)/(mid_-min_)
                    features.append(f'{a}_{b}_{c}_imb2')

    return df[features]

In [3]:
if TRAINING:
    # load dataset 
    df_train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
    df_features = generate_features(df_train)
    
    X = df_features.values
    Y = df_train['target'].values
    
    
    X = X[np.isfinite(Y)]
    Y = Y[np.isfinite(Y)]
    index = np.arange(len(X))

In [4]:
import lightgbm as lgb 
import joblib
import os


models = []

# test to train ratio
N_fold = 5

os.system('mkdir models')

#model_path ='/kaggle/input/testing_data_set/models'
model_path ='/kaggle/input/optimize'

def train(model_dict, modelname='lgbm'):
    if TRAINING:
        model = model_dict[modelname]
        model.fit(X[index%N_fold!=i], Y[index%N_fold!=i],
          eval_set=(X[index%N_fold==i], Y[index%N_fold==i]),
          early_stopping_rounds=100,
          verbose = 10
         )
        models.append(model)
        joblib.dump(model, f'./models/{modelname}_{i}.model')
    else:
        models.append(joblib.load(f'{model_path}/{modelname}_{i}.model'))


        
model_dict = {
    'lgbm': lgb.LGBMRegressor(objective='regression_l1',
                           colsample_bytree=0.7019911744574896,
                           learning_rate=0.022635758411078528,
                           max_bin=511,
                           min_child_samples=2,
                           n_estimators=4797,
                           num_leaves=122,
                           reg_alpha=0.004252223402511765,
                           reg_lambda=0.11288241427227624
                          ),
}
for i in range(N_fold):
    train(model_dict, 'lgbm')

In [5]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [6]:
for (test, revealed_targets, sample_prediction) in iter_test:
    feat = generate_features(test)
    
    sample_prediction['target'] = np.mean([model.predict(feat) for model in models], 0)
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
